Universidad Torcuato Di Tella

Licenciatura en Tecnología Digital\
**Tecnología Digital VI: Inteligencia Artificial**


In [3]:
import os
import torch
import torchaudio
import tarfile
import wandb
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torch.utils.data import Dataset
from torchaudio.datasets import GTZAN
from torch.utils.data import DataLoader
import torchaudio.transforms as tt
from torch.utils.data import random_split
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"


# TP3: Encodeador de música



## Orden de pasos

0. Elijan GPU para que corra mas rapido (RAM --> change runtime type --> T4 GPU)
1. Descargamos el dataset y lo descomprimimos en alguna carpeta en nuestro drive.
2. Conectamos la notebook a gdrive y seteamos data_dir con el path a los archivos.
3. Visualización de los archivos
4. Clasificación
5. Evaluación




In [4]:
project_name='TP3-TD6'
username = "sansonmariano-universidad-torcuato-di-tella"
wandb.login(key="d2875c91a36209496ee81454cccd95ebe3dc948d")
wandb.init(project = project_name, entity = username)

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: jaimeamigo (sansonmariano-universidad-torcuato-di-tella). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\45235544\_netrc


In [5]:
random_seed = 42

torch.manual_seed(random_seed)

# Definir parámetros
samplerate = 22050
data_dir = './genres_5sec'

init_batch_size = 20
init_num_epochs = 10
init_lr = 0.0005

In [6]:
# Función para parsear géneros
def parse_genres(fname):
    parts = fname.split('/')[-1].split('.')[0]
    return parts

# Definir la clase del dataset
class MusicDataset(Dataset):
    def __init__(self, root):
        super().__init__()
        self.root = root
        self.files = []
        for c in os.listdir(root):
            self.files += [os.path.join(root, c, fname) for fname in os.listdir(os.path.join(root, c)) if fname.endswith('.wav')]
        self.classes = list(set(parse_genres(fname) for fname in self.files))

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        fpath = self.files[idx]
        genre = parse_genres(fpath)
        class_idx = self.classes.index(genre)
        audio = torchaudio.load(fpath)[0]
        return audio, class_idx

In [7]:
import torch
import torchaudio
from torch.utils.data import DataLoader, Subset, Dataset
from sklearn.model_selection import StratifiedShuffleSplit

# Function to normalize the audio data across the dataset
def normalize_audio_data(dataset):
    """
    Normalize the dataset by calculating the mean and standard deviation
    of all audio samples and then applying standardization.
    """
    all_data = []
    for i in range(len(dataset)):
        audio, _ = dataset[i]
        all_data.append(audio)
    stacked_data = torch.cat(all_data, dim=1)  # Concatenate along the time dimension for mean/std calculation
    mean = stacked_data.mean()
    std = stacked_data.std()

    # Apply normalization to each sample
    normalized_data = []
    for i in range(len(dataset)):
        audio, label = dataset[i]
        normalized_audio = (audio - mean) / std
        normalized_data.append((normalized_audio, label))

    return normalized_data

# Function to create stratified train, validation, and test DataLoaders
def create_dataloaders(dataset, batch_size, test_size=0.3, val_size=0.5, random_state=42):
    """
    Normalizes the dataset, splits it into train, validation, and test subsets,
    and returns corresponding DataLoaders.
    """
    # Normalize the dataset
    normalized_data = normalize_audio_data(dataset)

    # Extract labels for stratified splitting
    labels = [label for _, label in normalized_data]

    # Stratified split: train and temporary (val+test) split
    split = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    for train_idx, temp_idx in split.split(range(len(normalized_data)), labels):
        train_dataset = Subset(normalized_data, train_idx)
        temp_dataset = Subset(normalized_data, temp_idx)

    # Stratified split on temp data: validation and test split
    val_test_labels = [labels[i] for i in temp_idx]
    split = StratifiedShuffleSplit(n_splits=1, test_size=val_size, random_state=random_state)
    for val_idx, test_idx in split.split(temp_idx, val_test_labels):
        val_dataset = Subset(normalized_data, [temp_idx[i] for i in val_idx])
        test_dataset = Subset(normalized_data, [temp_idx[i] for i in test_idx])

    # Create DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

    return train_loader, val_loader, test_loader

# Usage example with MusicDataset
dataset = MusicDataset(data_dir)
batch_size = 20
train_loader, val_loader, test_loader = create_dataloaders(dataset, batch_size)


In [8]:
list_files=os.listdir(data_dir)

classes = []

for file in list_files:

  name='{}/{}'.format(data_dir,file)

  if os.path.isdir(name):

    classes.append(file)

### 3. Visualización de los archivos

In [9]:
def audio_to_spectrogram(waveform):
    # Ensure the waveform is in the correct shape
    if len(waveform.shape) == 1:
        waveform = waveform.unsqueeze(0)
    
    # Convert the waveform to a spectrogram
    spectrogram = tt.Spectrogram()(waveform)
    return spectrogram

def process_dataloader_to_spectrograms(dataloader):
    spectrograms = []
    
    for batch in dataloader:
        # Assuming the batch is a tuple (waveforms, labels) and waveforms are the audio data
        waveforms, labels = batch
        
        # Process each waveform in the batch
        batch_spectrograms = [audio_to_spectrogram(waveform) for waveform in waveforms]
        
        # Append to the list of spectrograms
        spectrograms.append((torch.stack(batch_spectrograms), labels))
    
    return spectrograms

In [10]:
input_channels = 1  # for RGB images, or 1 for grayscale
num_classes = 10    # depends on your specific classification task

# Move the model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
train_spectogram = process_dataloader_to_spectrograms(train_loader)
val_spectogram = process_dataloader_to_spectrograms(val_loader)
test_spectogram = process_dataloader_to_spectrograms(test_loader)

### Ejercicio 1

Modelo que recibe el tamaño de la capa y la cantidad de capas

In [10]:
# Ajuste en la clase del modelo
class ExperimentNN(nn.Module):
    def __init__(self, input_size, num_classes, layer_size, layers):
        super(ExperimentNN, self).__init__()
        self.fc_layers = nn.ModuleList()
        self.dropout = nn.Dropout(p=0.5)  # Dropout layer with 50% probability

        for layer in range(layers - 1):
            if layer == 0:
                self.fc_layers.append(nn.Linear(input_size, layer_size))
            else:
                self.fc_layers.append(nn.Linear(layer_size, layer_size))
        
        # Output layer
        self.fc_layers.append(nn.Linear(layer_size, num_classes))

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Aplanar la onda de audio
        x = (x - x.mean()) / x.std()  # Normalización
        for fc in self.fc_layers[:-1]:  # Skip last layer
            x = self.dropout(F.relu(fc(x)))  # ReLU + Dropout
        x = self.fc_layers[-1](x)  # Output layer (no activation)
        return x

In [11]:
import gc
import copy

def train_model(model, criterion, optimizer, scheduler, epochs, train_loader, val_loader, device):
    best_loss = float("inf")
    best_model_state = None  # To store the best model's state

    for epoch in range(epochs):
        # Training loop
        model.train()
        train_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)
        print(f"Epoch {epoch+1}/{epochs} - Train loss: {train_loss:.4f}")

        # Validation step
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
        val_loss /= len(val_loader)
        print(f"Validation loss: {val_loss:.4f}")

        # Check if the current validation loss is the best
        if val_loss < best_loss:
            best_loss = val_loss
            best_model_state = copy.deepcopy(model.state_dict())  # Save model state
            print(f"New best validation loss: {best_loss:.4f}")

        # Update learning rate with scheduler
        scheduler.step(val_loss)
        
        # Clear cache and collect garbage
        gc.collect()
        torch.cuda.empty_cache()

    # Load the best model's state into the model and return it
    model.load_state_dict(best_model_state)
    return best_loss, model  # Return the final best model


In [13]:

input_size = samplerate * 5

num_classes = len(dataset.classes)  # Número de clases (géneros musicales)

# Define nuevos valores de hiperparámetros para experimentar
layers_list = [2, 3, 5, 7]       # Pruebas con más capas
sizes_list = [32, 64, 128, 256]  # Pruebas con más unidades en cada capa

best_val_loss = float("inf")
best_val_accuracy = 0
learning_rate = 0.0005
weight_decay = 1e-4
best_model = None

# Loop de experimentación
for layers in layers_list:
    for size in sizes_list:
        print(f"Experimentando con {layers} capas y {size} unidades por capa...")
        
        # Inicializar el modelo con la configuración actual
        model = ExperimentNN(input_size,
                             num_classes,
                             size,
                             layers).to(device)

        # Definir el criterio y optimizador con los pesos de las clases
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

        # Define el scheduler
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

        # Llama a train_model pasándole el scheduler
        validation_loss, temp_model = train_model(
            model, criterion, optimizer, scheduler=scheduler, epochs=15,
            train_loader=train_loader, val_loader=val_loader, device=device
        )

        # Calcular precisión de validación
        temp_model.eval()
        correct = 0
        total = len(val_loader.dataset)
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = temp_model(inputs)  # Use temp_model here
                predicted = outputs.argmax(dim=1)
                correct += (predicted == labels).sum().item()
        val_accuracy = 100 * correct / total

        print(f"Loss de validación para {layers} capas y {size} unidades: {validation_loss}")
        print(f"Precisión de validación: {val_accuracy:.2f}%")

        # Guardar el modelo con mejor precisión y menor pérdida
        if val_accuracy > best_val_accuracy or (val_accuracy == best_val_accuracy and validation_loss < best_val_loss):
            best_val_loss = validation_loss
            best_val_accuracy = val_accuracy
            best_model = temp_model  # Store the model with the best configuration
            print(f"Nuevo mejor modelo encontrado con {layers} capas y {size} unidades.")

print(f"Mejor modelo: {layers} capas y {size} unidades, con precisión de validación de {best_val_accuracy:.2f}% y pérdida de validación de {best_val_loss:.4f}")


Experimentando con 2 capas y 32 unidades por capa...
Epoch 1/15 - Train loss: 3.6866
Validation loss: 3.6617
New best validation loss: 3.6617
Epoch 2/15 - Train loss: 2.8721
Validation loss: 4.1420
Epoch 3/15 - Train loss: 2.3004
Validation loss: 4.1272
Epoch 4/15 - Train loss: 1.7740
Validation loss: 4.3407
Epoch 5/15 - Train loss: 1.5943
Validation loss: 4.5019
Epoch 6/15 - Train loss: 1.2865
Validation loss: 4.4943
Epoch 7/15 - Train loss: 1.1259
Validation loss: 4.8583
Epoch 8/15 - Train loss: 1.0920
Validation loss: 5.0810
Epoch 9/15 - Train loss: 1.0538
Validation loss: 5.0647
Epoch 10/15 - Train loss: 0.7920
Validation loss: 5.0582
Epoch 11/15 - Train loss: 0.9502
Validation loss: 5.3087
Epoch 12/15 - Train loss: 0.8757
Validation loss: 4.8651
Epoch 13/15 - Train loss: 0.8179
Validation loss: 5.0272
Epoch 14/15 - Train loss: 0.7303
Validation loss: 4.9836
Epoch 15/15 - Train loss: 0.6440
Validation loss: 5.3749
Loss de validación para 2 capas y 32 unidades: 3.6616984605789185
Pr

In [ ]:
# Evaluación final en el conjunto de prueba
print("Evaluando el mejor modelo en el conjunto de prueba...")
best_model.eval()
test_loss = 0.0
correct = 0
total = len(test_loader.dataset)
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = best_model(inputs)

        loss = criterion(outputs, labels)
        test_loss += loss.item()

        predicted = outputs.argmax(dim=1)
        correct += (predicted == labels).sum().item()

test_loss /= len(test_loader)
test_accuracy = 100 * correct / total

print(f"Loss en el conjunto de prueba: {test_loss:.4f}")
print(f"Precisión en el conjunto de prueba: {test_accuracy:.2f}%")

### 4. Clasificación

In [14]:
class CNN(nn.Module):
    def __init__(self, input_channels, num_classes, conv_layers_config):
        super(CNN, self).__init__()

        # Initialize the list to hold the convolutional layers
        self.conv_layers = nn.ModuleList()

        # Initialize the number of input channels for the first layer
        in_channels = input_channels

        # Dynamically create convolutional layers based on the configuration
        for (out_channels, kernel_size, stride, padding) in conv_layers_config:
            self.conv_layers.append(nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding))
            in_channels = out_channels  # Update in_channels for the next layer

        # Calculate the size after convolution and pooling to define the fully connected layer
        # Assuming pooling reduces the size by a factor of 2 at each layer
        self.pool = nn.MaxPool2d(2, 2)
        
        # Get the final feature map size (after all conv and pooling layers)
        self.final_feature_map_size = self._get_conv_output_size(input_channels, conv_layers_config)
        
        # Define 9 fully connected layers with 256 nodes each
        self.fc_layers = nn.ModuleList()
        self.fc_layers.append(nn.Linear(self.final_feature_map_size, 64))  # First fully connected layer
        for _ in range(2):  # Add 8 more fully connected layers with 256 nodes
            self.fc_layers.append(nn.Linear(64, 64))
        
        # Output layer
        self.fc_out = nn.Linear(64, num_classes)  # Output layer for classification
        
    def _get_conv_output_size(self, input_channels, conv_layers_config):
        # Sample input size (height x width) to calculate the final feature map size
        # You can adjust these values based on your actual input size
        height = 201  # Replace with your actual input height
        width = 552   # Replace with your actual input width
        
        # Apply each convolutional and pooling layer
        for (out_channels, kernel_size, stride, padding) in conv_layers_config:
            height = (height + 2 * padding - kernel_size) // stride + 1
            width = (width + 2 * padding - kernel_size) // stride + 1
            height = height // 2  # Max pooling halves the height
            width = width // 2    # Max pooling halves the width
        
        # Return the total number of features after all convolutional and pooling layers
        return out_channels * height * width

    def forward(self, x):
        # Apply each convolutional layer followed by ReLU and pooling
        for conv_layer in self.conv_layers:
            x = F.relu(conv_layer(x))
            x = self.pool(x)
        
        # Flatten the output before passing it to the fully connected layers
        x = x.view(x.size(0), -1)  # Flatten the feature map

        # Apply the fully connected layers
        for fc in self.fc_layers:
            x = F.relu(fc(x))
        
        # Output layer (classification)
        x = self.fc_out(x)

        return x


In [31]:
import copy

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, device="cpu"):
    """
    Train the model and select the best gradients based on validation loss.
    """
    model.train()  # Set the model to training mode
    best_val_loss = float("inf")
    best_gradients = None  # To store gradients with the lowest validation loss
    best_accuracy = 0

    for epoch in range(num_epochs):
        running_loss = 0.0
        print(f"\nStarting Epoch {epoch+1}/{num_epochs}")
        
        # Training loop
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            # Accumulate training loss
            running_loss += loss.item()
            
            # Print loss for every batch
            if (i + 1) % 10 == 0 or (i + 1) == len(train_loader):
                print(f"  Batch {i+1}/{len(train_loader)} - Loss: {loss.item():.4f}")
        
        # Calculate average training loss for the epoch
        avg_train_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch+1} Training completed. Average Loss: {avg_train_loss:.4f}")
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        correct = 0
        total = len(val_loader)
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
        
        accuracy = 100 * correct / total

        avg_val_loss = val_loss / total
        print(f"Epoch {epoch+1} Validation completed. Average Loss: {avg_val_loss:.4f}")
        
        # If this epoch has the best (lowest) validation loss, save the gradients
        if (accuracy > best_accuracy) or ((accuracy == best_accuracy) and (avg_val_loss < best_val_loss)):
            best_val_loss = avg_val_loss
            best_accuracy = accuracy
            # Capture gradients
            best_gradients = {name: param.grad.clone() for name, param in model.named_parameters() if param.grad is not None}
            print(f"New best gradients stored for epoch {epoch+1} with validation loss {best_val_loss:.4f} and accuracy {accuracy}%")
        
        # Set model back to training mode
        model.train()

    print("\nTraining complete.")
    
    return best_gradients

def test_model_configuration(model, test_loader, criterion, device):
    """
    Evaluate the model on the test set.
    """
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    correct = 0
    total = 0  # Total number of samples processed

    # Ensure no gradient computation during evaluation
    with torch.no_grad():
        for inputs, labels in test_loader:  # Assuming test_loader is the correct DataLoader for your test set
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs)

            # Compute the loss
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            # Get the predicted class labels (the one with the highest logit)
            _, predicted = torch.max(outputs.data, 1)

            # Accumulate total samples
            total += labels.size(0)

            # Accumulate correct predictions
            correct += (predicted == labels).sum().item()

    # Calculate average test loss
    test_loss /= len(test_loader)

    # Calculate accuracy
    accuracy = 100 * correct / total

    return test_loss, accuracy

def test_multiple_configurations(train, val, test, criterion, device, num_epochs=10):
    """
    Test multiple model configurations, applying the best gradients for evaluation,
    and save the best model based on accuracy or validation loss.
    """
    # Different configurations for the CNN model
    configurations = [
        [(32, 3, 1, 1), (64, 3, 1, 1), (128, 3, 1, 1)],  # Configuration 1
        [(32, 5, 1, 2), (64, 5, 1, 2)],                  # Configuration 2
        [(16, 3, 1, 1), (32, 3, 1, 1), (64, 3, 1, 1)],   # Configuration 3
        [(64, 3, 1, 1), (128, 3, 1, 1), (256, 3, 1, 1)]  # Configuration 4
    ]
    
    best_model = None
    best_accuracy = 0.0
    best_loss = float("inf")
    
    for idx, conv_layers_config in enumerate(configurations):
        print(f"\nTesting Configuration {idx + 1} with convolutional layers: {conv_layers_config}")
        
        # Initialize the model with the current configuration
        model = CNN(input_channels=1, num_classes=10, conv_layers_config=conv_layers_config)
        model.to(device)
        
        # Initialize optimizer (e.g., Adam) and train the model
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        
        # Train the model on the training set
        best_gradients = train_model(model, train, val, criterion, optimizer, num_epochs, device)
        
        # Apply the best gradients to the model parameters
        if best_gradients is not None:
            with torch.no_grad():
                for name, param in model.named_parameters():
                    if name in best_gradients and param.grad is not None:
                        param.grad.copy_(best_gradients[name])
        
        # Evaluate the model on the test set with best gradients applied
        test_loss, accuracy = test_model_configuration(model, test, criterion, device)
        
        # Print the results for the current configuration
        print(f"Configuration {idx + 1} Test Loss: {test_loss:.4f}, Test Accuracy: {accuracy:.2f}%")
        
        # Save the best model based on accuracy; if accuracy is the same, use loss as the tie-breaker
        if accuracy > best_accuracy or (accuracy == best_accuracy and test_loss < best_loss):
            best_accuracy = accuracy
            best_loss = test_loss
            best_model = copy.deepcopy(model)
            print(f"New best model found with accuracy: {best_accuracy:.2f}% and loss: {best_loss:.4f}")

    print(f"\nBest Model Test Accuracy: {best_accuracy:.2f}% with Loss: {best_loss:.4f}")
    return best_loss, best_model

In [25]:
# Assume 'test_loader' is the DataLoader for your test set, and 'criterion' is the loss function (e.g., CrossEntropyLoss)
best_model = test_multiple_configurations(train_spectogram, val_spectogram, test_spectogram, criterion, device, 10)


Testing Configuration 1 with convolutional layers: [(32, 3, 1, 1), (64, 3, 1, 1), (128, 3, 1, 1)]

Starting Epoch 1/10
  Batch 10/35 - Loss: 2.3256
  Batch 20/35 - Loss: 2.2824
  Batch 30/35 - Loss: 2.3461
  Batch 35/35 - Loss: 2.2804
Epoch 1 Training completed. Average Loss: 27.9878
Epoch 1 Validation completed. Average Loss: 2.3046
New best gradients stored for epoch 1 with validation loss 2.3046

Starting Epoch 2/10
  Batch 10/35 - Loss: 2.2523
  Batch 20/35 - Loss: 2.1388
  Batch 30/35 - Loss: 2.0734
  Batch 35/35 - Loss: 1.9359
Epoch 2 Training completed. Average Loss: 2.1825
Epoch 2 Validation completed. Average Loss: 2.3035
New best gradients stored for epoch 2 with validation loss 2.3035

Starting Epoch 3/10
  Batch 10/35 - Loss: 2.0717
  Batch 20/35 - Loss: 2.0913
  Batch 30/35 - Loss: 1.8698
  Batch 35/35 - Loss: 1.2375
Epoch 3 Training completed. Average Loss: 1.9165
Epoch 3 Validation completed. Average Loss: 2.5949

Starting Epoch 4/10
  Batch 10/35 - Loss: 1.2280
  Batch

In [27]:
print(f"Loss: {best_model[0]} - Model: {best_model[1]}")

Loss: 6.56303671002388 - Model: CNN(
  (conv_layers): ModuleList(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc_layers): ModuleList(
    (0): Linear(in_features=220800, out_features=64, bias=True)
    (1-2): 2 x Linear(in_features=64, out_features=64, bias=True)
  )
  (fc_out): Linear(in_features=64, out_features=10, bias=True)
)


In [29]:
cnn_best_model = best_model[1]

test_model_configuration(cnn_best_model,test_spectogram,criterion, device)

(6.56303671002388, 30.201342281879196)

In [33]:
class different_act_CNN(nn.Module):
    def __init__(self, input_channels=1, num_classes=10, activation_function="relu"):
        super(different_act_CNN, self).__init__()

        # Define the convolutional layers (fixed as per example)
        self.conv_layers = nn.ModuleList([
            nn.Conv2d(input_channels, 32, kernel_size=3, stride=1, padding=1),  # Conv layer 1
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),              # Conv layer 2
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)              # Conv layer 3
        ])
        
        # Max Pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Calculate the output feature size after conv layers and pooling
        self.final_feature_map_size = self._get_conv_output_size(input_channels)

        # Define fully connected layers (fixed as per example)
        self.fc_layers = nn.ModuleList([
            nn.Linear(self.final_feature_map_size, 64),  # First fully connected layer
            nn.Linear(64, 64),                           # Second fully connected layer
            nn.Linear(64, 64)                            # Third fully connected layer
        ])

        # Output layer
        self.fc_out = nn.Linear(64, num_classes)

        # Set activation function
        self.activation = self._get_activation_function(activation_function)

    def _get_conv_output_size(self, input_channels):
        # Sample input size (height x width)
        height, width = 201, 552  # Replace with actual input height and width

        # Pass through convolutional and pooling layers to determine final size
        for layer in self.conv_layers:
            height = (height + 2 * layer.padding[0] - layer.kernel_size[0]) // layer.stride[0] + 1
            width = (width + 2 * layer.padding[1] - layer.kernel_size[1]) // layer.stride[1] + 1
            height //= 2  # Max pooling halves the height
            width //= 2   # Max pooling halves the width

        # Output feature map size
        return height * width * 128

    def _get_activation_function(self, activation_function):
        # Map the string to the appropriate activation function
        if activation_function == "relu":
            return F.relu
        elif activation_function == "leaky_relu":
            return F.leaky_relu
        elif activation_function == "tanh":
            return torch.tanh
        elif activation_function == "sigmoid":
            return torch.sigmoid
        elif activation_function == "softmax":
            return F.softmax
        elif activation_function == "elu":
            return F.elu
        elif activation_function == "selu":
            return F.selu
        elif activation_function == "gelu":
            return F.gelu
        elif activation_function == "swish":
            return lambda x: x * torch.sigmoid(x)  # Swish activation: x * sigmoid(x)
        elif activation_function == "hard_sigmoid":
            return F.hardsigmoid
        else:
            raise ValueError(f"Unsupported activation function: {activation_function}")

    def forward(self, x):
        # Forward pass through conv layers with the selected activation function and pooling
        for conv_layer in self.conv_layers:
            x = self.activation(conv_layer(x))
            x = self.pool(x)
        
        # Flatten the output from conv layers
        x = x.view(x.size(0), -1)

        # Forward pass through fully connected layers with the selected activation function
        for fc_layer in self.fc_layers:
            x = self.activation(fc_layer(x))
        
        # Final output layer
        x = self.fc_out(x)

        return x

In [31]:
# Function to test different activation functions
def test_multiple_activation_functions(train, val, test, criterion, device, num_epochs=10):
    activation_functions = ["relu", "leaky_relu", "tanh", "sigmoid", "softmax", "elu", "selu", "gelu", "swish", "hard_sigmoid"]
    
    best_model = None
    best_accuracy = 0.0
    best_loss = float("inf")
    
    for activation_function in activation_functions:
        print(f"\nTesting activation function: {activation_function}")
        
        model = different_act_CNN(input_channels=1, num_classes=10, activation_function=activation_function)
        model.to(device)
        
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        
        best_gradients = train_model(model, train, val, criterion, optimizer, num_epochs, device)
        
        if best_gradients is not None:
            with torch.no_grad():
                for name, param in model.named_parameters():
                    if name in best_gradients and param.grad is not None:
                        param.grad.copy_(best_gradients[name])
        
        test_loss, accuracy = test_model_configuration(model, test, criterion, device)
        
        print(f"Activation function: {activation_function} Test Loss: {test_loss:.4f}, Test Accuracy: {accuracy:.2f}%")
        
        if accuracy > best_accuracy or (accuracy == best_accuracy and test_loss < best_loss):
            best_accuracy = accuracy
            best_loss = test_loss
            best_model = copy.deepcopy(model)
            print(f"New best model found with accuracy: {best_accuracy:.2f}% and loss: {best_loss:.4f}")

    print(f"\nBest Model Test Accuracy: {best_accuracy:.2f}% with Loss: {best_loss:.4f}")
    return best_loss, best_model

In [34]:
# Assume 'test_loader' is the DataLoader for your test set, and 'criterion' is the loss function (e.g., CrossEntropyLoss)
best_loss, best_model = test_multiple_activation_functions(train_spectogram, val_spectogram, test_spectogram, criterion, device, 10)


Testing activation function: relu

Starting Epoch 1/10
  Batch 10/35 - Loss: 3.2699
  Batch 20/35 - Loss: 2.2649
  Batch 30/35 - Loss: 2.3546
  Batch 35/35 - Loss: 2.2916
Epoch 1 Training completed. Average Loss: 21.6914
Epoch 1 Validation completed. Average Loss: 2.2721
New best gradients stored for epoch 1 with validation loss 2.2721

Starting Epoch 2/10
  Batch 10/35 - Loss: 2.3187
  Batch 20/35 - Loss: 2.2398
  Batch 30/35 - Loss: 2.2575
  Batch 35/35 - Loss: 2.2021
Epoch 2 Training completed. Average Loss: 2.1952
Epoch 2 Validation completed. Average Loss: 2.2713
New best gradients stored for epoch 2 with validation loss 2.2713

Starting Epoch 3/10
  Batch 10/35 - Loss: 2.2286
  Batch 20/35 - Loss: 2.0982
  Batch 30/35 - Loss: 2.0084
  Batch 35/35 - Loss: 1.9822
Epoch 3 Training completed. Average Loss: 2.0515
Epoch 3 Validation completed. Average Loss: 2.3029

Starting Epoch 4/10
  Batch 10/35 - Loss: 1.5455
  Batch 20/35 - Loss: 1.8817
  Batch 30/35 - Loss: 1.7527
  Batch 35/35

C:\Users\45235544\AppData\Local\Temp\ipykernel_17480\3811288260.py:73: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.activation(conv_layer(x))
C:\Users\45235544\AppData\Local\Temp\ipykernel_17480\3811288260.py:81: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.activation(fc_layer(x))


  Batch 10/35 - Loss: 2.3005
  Batch 20/35 - Loss: 2.3043
  Batch 30/35 - Loss: 2.3177
  Batch 35/35 - Loss: 2.3132
Epoch 1 Training completed. Average Loss: 2.3041
Epoch 1 Validation completed. Average Loss: 2.3033
New best gradients stored for epoch 1 with validation loss 2.3033

Starting Epoch 2/10
  Batch 10/35 - Loss: 2.2997
  Batch 20/35 - Loss: 2.3035
  Batch 30/35 - Loss: 2.3161
  Batch 35/35 - Loss: 2.3108
Epoch 2 Training completed. Average Loss: 2.3036
Epoch 2 Validation completed. Average Loss: 2.3032
New best gradients stored for epoch 2 with validation loss 2.3032

Starting Epoch 3/10
  Batch 10/35 - Loss: 2.2996
  Batch 20/35 - Loss: 2.3027
  Batch 30/35 - Loss: 2.3146
  Batch 35/35 - Loss: 2.3086
Epoch 3 Training completed. Average Loss: 2.3034
Epoch 3 Validation completed. Average Loss: 2.3030
New best gradients stored for epoch 3 with validation loss 2.3030

Starting Epoch 4/10
  Batch 10/35 - Loss: 2.2995
  Batch 20/35 - Loss: 2.3021
  Batch 30/35 - Loss: 2.3134
  Ba